In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [126]:
import torch 

from DEPO.depo import depo_best
from utils.model import load_checkpoint
from utils.metrics import mAA, rotation_angle_distance_batch, vector_angle_distance_batch, mAA_max, mAA_sep

from data.seven_scenes.utils_7scenes import SevenScenesEvalDataset
from training.seven_scenes_eval import test
from torch.utils.data import DataLoader
from pathlib import Path

import numpy as np
from numpy.linalg import norm

import pickle
import os.path as osp
from pathlib import Path
from tqdm.auto import tqdm

import pandas as pd

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## 0. Data

In [133]:
pairs = pd.read_csv('/home/project/data/7scenes/NN_7scenes.txt', header=None, sep=' ')

absent = []

for i in tqdm(range(pairs.shape[0])):
    
    scenes_dicitionary = {
            0: 'chess', 
            1: 'fire',
            2: 'heads',
            3: 'office',
            4: 'pumpkin',
            5: 'redkitchen',
            6: 'stairs'} 
    
    name_0 = osp.join('/home/project/data/7scenes/', scenes_dicitionary[pairs.iloc[i][2]], pairs.iloc[i][0][1:])
    name_1 = osp.join('/home/project/data/7scenes/', scenes_dicitionary[pairs.iloc[i][2]], pairs.iloc[i][1][1:])
    
    if (not Path(name_0).is_file()) or (not Path(name_1).is_file()):
        absent.append(i)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 85000/85000 [00:23<00:00, 3604.19it/s]


In [141]:
pairs.drop(absent, axis=0, inplace=True) #problems in heads sequence

In [142]:
pairs.to_csv('/home/project/data/7scenes/NN_7_scenes_corrected.txt', header=False, index=False, sep=' ')

## 1. Metrics

In [8]:
metrics = {}

if Path('/home/project/code/results/7scenes/metrics_depo').is_file():
    with open('/home/project/code/results/7scenes/metrics_depo', 'rb') as f:
        metrics = pickle.load(f)

### 1.1 DEPO-C-f: PVT-L + QT-B + Spatial pred. (final), without FT

In [9]:
checkpoint = load_checkpoint(
    '/home/project/code/src/weights/flow_and_pose_best_abs_0.pth',
    device)

In [10]:
model = depo_best(calculate_flow=False)
model.load_state_dict(checkpoint['model'])
model.to(device);

In [145]:
scenes = {
    'chess': 0,
    'fire' : 1,
    'heads': 2,
    'office': 3,
    'pumpkin': 4,
    'redkitchen': 5,
    'stairs': 6
}   

for scene_name, scene_id in scenes.items():
    test_data = SevenScenesEvalDataset(root_dir='/home/project/data/7scenes',
                                       pairs_path='/home/project/data/7scenes/NN_7_scenes_corrected.txt', scene=scene_id)
    test_loader = DataLoader(test_data, batch_size=8, shuffle=False, drop_last=False, pin_memory=True, num_workers=4)
    results = test(model, test_loader, device)
    
    metrics[scene_name] = {
        'R_angle_distances': rotation_angle_distance_batch(results['R_gt'], results['R']),
        't_angle_distances': vector_angle_distance_batch(results['t_gt'], results['t']),
        't_distances_l2': norm(results['t_gt'] - results['t'], ord=2, axis=1),
        'R_gt_distances': rotation_angle_distance_batch(results['R_gt'], np.tile(np.eye(3, 3), (results['R_gt'].shape[0], 1, 1))),
        't_gt_distances': norm(results['t_gt'], ord=2, axis=1)
    }

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 585/585 [02:15<00:00,  4.31it/s]


### Saving metrics

In [170]:
with open('/home/project/code/results/7scenes/metrics_before_ft', 'wb') as f:
    pickle.dump(metrics, f)

## 2. Table form

In [169]:
df = {
    ('Method', 1): [None],
    ('Chess', '$R_e^{0.5}$'): None,
    ('Chess', '$t_e^{0.5}$'): None,
    ('Fire', '$R_e^{0.5}$'): None,
    ('Fire', '$t_e^{0.5}$'): None,
    ('Heads', '$R_e^{0.5}$'): None,
    ('Heads', '$t_e^{0.5}$'): None,
    ('Office', '$R_e^{0.5}$'): None,
    ('Office', '$t_e^{0.5}$'): None,
    ('Pumpkin', '$R_e^{0.5}$'): None,
    ('Pumpkin', '$t_e^{0.5}$'): None,
    ('Redkitchen', '$R_e^{0.5}$'): None,
    ('Redkitchen', '$t_e^{0.5}$'): None,
    ('Stairs', '$R_e^{0.5}$'): None,
    ('Stairs', '$t_e^{0.5}$'): None,
    ('Average', '$R_e^{0.5}$'): None,
    ('Average', '$t_e^{0.5}$'): None,
}


df = pd.DataFrame(df)

df.columns = pd.MultiIndex.from_tuples(df.columns)

df.loc[0] = ['RelPoseNet', 8.149, 0.261, 12.327, 0.382, 12.414, 0.234, 9.045, 0.307, 9.362, 0.352, 9.636, 0.407, 12.259, 0.367, 10.456, 0.342]
df.loc[1] = ['8Point', 3.338, 0.123, 7.819, 0.270, 10.384, 0.172, 4.059, 0.181, 3.682, 0.187, 4.145, 0.210, 5.018, 0.263, 5.492, 0.214]
df.loc[2] = ['Map-free rep.', 5.835, 0.154, 12.207, 0.334, 12.542, 0.191, 7.319, 0.237, 6.647, 0.269, 7.093, 0.267, 10.167, 0.313, 8.830, 0.268]
df.loc[3] = ['Map-free Scannet', 5.445, 0.144, 5.110, 0.149, 3.406, 0.112, 3.518, 0.13, 5.506, 0.266, 3.754, 0.185, 3.817, 0.152, 4.365, 0.166]
df.loc[4] = ['GRelPose', 4.463, 0.147, 7.206, 0.195, 7.254, 0.128, 4.653, 0.167, 4.797, 0.225, 4.543, 0.203, 5.840, 0.238, 5.537, 0.186]
df.loc[5] = ['GRelPose-Scannet', 3.568, 0.115, 4.946, 0.142, 6.195, 0.123, 3.660, 0.140, 3.670, 0.168, 4.019, 0.149, 6.344, 0.225, 4.630, 0.158]

i=6
df.loc[i] = None
df.loc[i]['Method'] = 'DEPO-C-f: PVT-L + QT-B + Spatial pred., without FT'
total_t = []
total_r = []
for key, val in metrics.items():
    total_t.append(val["t_distances_l2"])
    total_r.append(val["R_angle_distances"])
    df.loc[i, (key.capitalize(), '$t_e^{0.5}$')] = np.median(val["t_distances_l2"])
    df.loc[i, (key.capitalize(), '$R_e^{0.5}$')] = np.median(val["R_angle_distances"])
total_t = np.concatenate(total_t)
total_r = np.concatenate(total_r)
# df.loc[i, ('Average', '$t_e^{0.5}$')] = np.median(total_t)
# df.loc[i, ('Average', '$R_e^{0.5}$')] = np.median(total_r)
#Hm

df.loc[i, ('Average', '$R_e^{0.5}$')] = df.iloc[i, 1:-2:2].mean()
df.loc[i, ('Average', '$t_e^{0.5}$')] = df.iloc[i, 2:-2:2].mean()




df.iloc[:, 1:] = df.iloc[:, 1:].astype('float32')
d = df.style.format(precision=3)
d.background_gradient(cmap="RdYlGn_r", axis=0)
    

/tmp/ipykernel_5436/1370051084.py:34: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[i] = None
/tmp/ipykernel_5436/1370051084.py:55: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 1:] = df.iloc[:, 1:].astype('float32')
